## 2단계 : 편의점 관련 feature 추출 및 Scoring

### # 편의점 feature 데이터 추출

In [ ]:
dong_cod_retail = pd.read_excel(data_path+'offer/02_혁신아이디어분야_카드소비데이터(신한카드)_데이터정의서.xlsx', sheet_name='(참고) 행정동코드')
dong_cod_retail = dong_cod_retail.set_index('행정동코드')
dong_cod_retail.head()

In [ ]:
card = pd.read_csv(data_path+'CARD_SPENDING_RESIDENT.txt', delimiter='	')
card = card[card.MCT_CAT_CD != 40]

card_2019 = card[card['STD_DD'] <= 20191231]
card_2019 = card_2019.groupby(['DONG_CD']).sum().drop(['MCT_CAT_CD','STD_DD','GU_CD', 'AGE_CD'], axis=1)

card_2020 = card[card['STD_DD'] > 20191231]
card_2020 = card_2020.groupby(['DONG_CD']).sum().drop(['MCT_CAT_CD','STD_DD','GU_CD', 'AGE_CD'], axis=1)

In [ ]:
card_rateOfChange = (card_2020 - card_2019) / card_2019
card_rateOfChange['행정동'] = 'Nan'
card_rateOfChange.to_csv(data_path+'행정동코드추출.csv')

In [ ]:
card_rateOfChange = pd.read_csv(data_path+'행정동코드추출_f.csv', encoding='CP949')
card_rateOfChange = card_rateOfChange.set_index('행정동')

In [ ]:
card_rateOfChange.index

In [ ]:
all_sheets = pd.read_excel(data_path+'untact service related features.xlsx', sheet_name=None)

In [ ]:
all_sheets.keys()

In [ ]:
csv_data = all_sheets['draft_cvs']
csv_data['card_rateOfChange'] = 'Nan'
csv_data.head(2)

In [ ]:
for i in range(len(csv_data)):
    
    try:
        tempDong = csv_data.iloc[i]['행정동']
        val = card_rateOfChange.loc[tempDong]['USE_AMT']
        #print(val)
    except:
        val = card_rateOfChange.loc['평균']['USE_AMT']
    csv_data.at[i,'card_rateOfChange'] = val

In [ ]:
card_rateOfChange.loc['평균']['USE_AMT']

In [ ]:
csv_data['card_rateOfChange'].describe()

In [ ]:
feature = csv_data[csv_data['Unnamed: 0'] == '서울시'].drop(['Unnamed: 0','구','동','주소','좌표'], axis=1)

In [ ]:
feature['범위 내 학교 (초중고?) : 학생증 청소년층'] = feature['범위 내 학교 (초중고?) : 학생증 청소년층']# * -1
feature['범위 내 편의점수'] = feature['범위 내 편의점수'] * -1
feature.head()

### # 특정 feature 만 추출

In [ ]:
use_col = feature[['행정동', '매장정보','범위 내 유흥업소','범위 내 편의점수', 
                   '범위 내 학교 (초중고?) : 학생증 청소년층', '범위 내 숙박시설', '범위 내 관광지', 'card_rateOfChange']]
use_col.columns = ['dong', 'name','ent_num','cvs_num', 'sch_num','acc_num','tou_num','rate_change']

In [ ]:
use_col['rate_change'].describe()

In [ ]:
print(use_col['rate_change'].min())
print(use_col['rate_change'].max())

In [ ]:
use_col.describe()

### # 최대최소 정규화

In [ ]:
scaled_data_minmax = pd.DataFrame(columns=use_col.columns)

# 최대최소 정규화
from sklearn.preprocessing import MinMaxScaler
for i in range(2,len(scaled_data_minmax.columns)):
    temp_values = use_col[use_col.columns[i]].values.reshape(-1,1)
    temp_scale = MinMaxScaler().fit_transform(temp_values)
    scaled_data_minmax[scaled_data_minmax.columns[i]] = temp_scale.reshape(-1)

    
scaled_data_minmax.iloc[:,:2] = use_col.iloc[:,:2].copy()

In [ ]:
scaled_data_minmax.describe()

### # Z-score 정규화

In [ ]:
scaled_data_zscore = pd.DataFrame(columns=use_col.columns)

# Z-score 정규화
from sklearn.preprocessing import StandardScaler
for i in range(2,len(scaled_data_zscore.columns)):
    temp_values = use_col[use_col.columns[i]].values.reshape(-1,1)
    temp_scale = StandardScaler().fit_transform(temp_values)
    scaled_data_zscore[scaled_data_zscore.columns[i]] = temp_scale.reshape(-1)

scaled_data_zscore.iloc[:,:2] = use_col.iloc[:,:2].copy()

In [ ]:
scaled_data_zscore.describe()

### # 가중치 부여 Scoring

In [ ]:
def getScoreCol(scaled_data):
    temp_data = scaled_data.copy()
    
    # feature 수
    num_feature = len(temp_data.columns)-2
    
    # 가중치
    weight = [1/num_feature for _ in range(num_feature)] # [0.2, 0.2, 0.2, 0.2, 0.2]
    
    # score column 초기화
    sum_col = 0
    for i in range(num_feature):
        sum_col += temp_data.iloc[:,i+2]*weight[i] # 혹시 이거 .iloc[i,]
    temp_data['score'] = sum_col
    
    return temp_data

In [ ]:
# minmax
result = getScoreCol(scaled_data_minmax)

# score가 제일 높은 편의점
result[result.score == result.score.max()]

In [ ]:
result

In [ ]:

# z-score  
result_z = getScoreCol(scaled_data_zscore)

# score가 제일 높은 편의점
result[result.score == result.score.max()]

In [ ]:
result_z.set_index('dong').sort_index()[['name','score']].sort_values(by='score', ascending=False)

In [ ]:
use_col[use_col['name'] == 'GS25을지로3가역점']